#Choose country and run


In [ ]:
# The options are 'KEN', 'CIV'. 'UGA', 'GHA', 'RWA'
country = 'KEN'

#Run this code afterwards

In [5]:
!pip install io
!pip install --upgrade pandas
!pip install xlsxwriter

ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io
  Using cached pandas-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
Using cached pandas-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.


  Using cached xlsxwriter-3.2.5-py3-none-any.whl.metadata (2.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 6.1 MB/s eta 0:00:00


In [1]:
# PART 1: IMPORTS
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from io import BytesIO
import xlsxwriter

# Note: This script requires 'xlsxwriter' to be installed for saving multi-sheet Excel files.
# You can install it via pip: pip install xlsxwriter

# =============================================================================
# PART 2: DATA LOADING AND PRE-PROCESSING
# =============================================================================
df = pd.read_csv('/content/central-tableau-export-2.0.csv')


countries = ['GHA', 'RWA', 'UGA', 'CIV', 'KEN']
df = df[df['Country'].isin(countries)]

df = df[~df['Round'].isin(['Onboarding', '6', '6.0']) & df['Round'].notna()]
df['Round'] = pd.to_numeric(df['Round'], errors='coerce').dropna()
df = df[df['Round'] % 1 == 0]
df['Round'] = df['Round'].astype(int)
df = df[df['Round'].isin([0, 1, 2, 3, 100, 102])]
df['Round'] = df['Round'].astype(str)
df = df.sort_values(by=['Country', 'Groupnr', 'Round'])

print("Initial 'Round' values found:", df['Round'].unique())

# =============================================================================
# PART 3: VARIABLE DEFINITIONS & INITIAL MAPPING
# =============================================================================
all_columns = [
    "Groupnr", "Round", "Country", "childmortality", "childmortalitytime", *[f"foodsecurity{i}" for i in range(1, 10)],
    *[f"foodsecurity{i}freq" for i in range(1, 10)],
    "fuelcooking", "sourcelighting", "watersource", "timewatersource_1", "timewatersourceunit", "Toiletfacility",
    "materialroof", "materialfloor", "materialwallsext",
    "assetsmatrix2_7", "assetsmatrix2_14", "assetsmatrix2_16", "assetsmatrix1_23", "assetsmatrix3_14",
    "assetsmatrix3_16", "assetsmatrix2_12", "assetsmatrix3_22",
    *[f"HHMschool_{n}" for n in range(1, 6)], *[f"HHMschoolnow_{n}" for n in range(1, 6)],
    *[f"HHMschoolcompl_{n}" for n in range(1, 6)], *[f"HHMage_1_{n}" for n in range(1, 21)], "school", "schoolcompleted",
    "savinghowmuch_1", "savinghowmuch_2", "savinghowmuch_3", "savingstotal_1", "debt", "debtamount_1", "debtnote",
    *[f"anxiety{i}" for i in range(1, 8)],
    "psychwellbeing_1", "psychwellbeing_3", "psychwellbeing_5", "psychwellbeing2_5", "jealousy", "jealousywhat",
    *[f"livestocknumbers_{i}" for i in [1, 13, 3, 4, 5, 6, 11, 8, 9, 7, 2, 10]], "assetsmatrix1_4", "assetsmatrix1_5",
    "assetsmatrix1_22", "assetsmatrix2_7", "assetsmatrix2_14",
    "assetsmatrix2_15", "assetsmatrix2_16", "assetsmatrix2_8", "assetsmatrix3_17", "assetsmatrix2_17",
    "assetsmatrix2_18", "assetsmatrix2_19", "assetsmatrix2_11",
    "assetsmatrix2_12", "assetsmatrix3_14", "assetsmatrix1_23", "assetsmatrix3_15", "assetsmatrix3_16",
    "assetsmatrix3_22", "assetsmatrix3_23", "occupationmain",
    "ownsland_scto", "meetings1", "moneywithdraw", "moneyproblems"
]

columns_available = [col for col in all_columns if col in df.columns]
df = df[columns_available]

def livestock_normal(val):
    if pd.isna(val) or val == 0: return 0
    val = int(val)
    if val in [1, 2, 3, 4, 5]: return str(val)
    elif 6 <= val <= 10: return '6-10'
    elif 11 <= val <= 20: return '11-20'
    elif val > 20: return '20+'
livestock_indices_normal = [1, 13, 3, 4, 8, 9, 7, 2, 10]
for i in livestock_indices_normal:
    col = f'livestocknumbers_{i}'
    if col in df.columns: df[col] = df[col].apply(livestock_normal)

def livestock_high(val):
    if pd.isna(val) or val == 0: return 0
    val = int(val)
    if val in [1, 2, 3, 4, 5]: return str(val)
    elif 6 <= val <= 10: return '6-10'
    elif 11 <= val <= 20: return '11-20'
    elif 21 <= val <= 50: return '21-50'
    elif val >= 51: return '51+'
    else: return 0

livestock_indices_high = [5,6,11]
for i in livestock_indices_high:
    col = f'livestocknumbers_{i}'
    if col in df.columns: df[col] = df[col].apply(livestock_high)

def mpi_water(row):
    if row['timewatersource_1'] > 30: return 1.0
    water = row['watersource']
    if water in [1, 2, 5, 7, 12]: return 0.0
    elif water in [4, 10]: return 0.3
    elif water in [3, 6]: return 0.6
    elif water in [8, 9, 11]: return 1.0
    else: return None
if 'timewatersource_1' in df.columns and 'watersource' in df.columns:
    df['MPI_water'] = df.apply(mpi_water, axis=1)

def MPI_fuel(row):
    fuel = row['fuelcooking']
    if fuel in [3, 4, 5, 6]: return 0.0
    elif fuel in [7]: return 0.3
    elif fuel in [2, 10]: return 0.6
    elif fuel in [1, 8, 9]: return 1.0
    else: return None
if 'fuelcooking' in df.columns:
    df['MPI_fuel'] = df.apply(MPI_fuel, axis=1)

def MPI_electricity(row):
    source = row['sourcelighting']
    if source in [1, 2, 4, 9, 15]: return 0.0
    elif source in [3]: return 0.3
    elif source in [5, 7, 8, 10, 13, 14]: return 0.6
    elif source in [6, 11, 12]: return 1.0
    else: return None
if 'sourcelighting' in df.columns:
    df['MPI_electricity'] = df.apply(MPI_electricity, axis=1)

def MPI_sanitation(row):
    toiletfacility = row['Toiletfacility']
    if toiletfacility in [1]: return 2
    elif toiletfacility in [6]: return 0.2
    elif toiletfacility in [5]: return 0.6
    elif toiletfacility in [3]: return 1.0
    else: return None
if 'Toiletfacility' in df.columns:
    df['MPI_sanitation'] = df.apply(MPI_sanitation, axis=1)

def MPI_floor(row):
    material = row['materialfloor']
    if material in [5, 6, 4, 9]: return 2
    elif material in [3, 8]: return 1
    elif material in [1, 2]: return 0
    else: return None

def MPI_roof(row):
    material = row['materialroof']
    if material in [4, 2, 3, 8]: return 2
    elif material in [5, 7]: return 1
    elif material in [1, 9]: return 0
    else: return None

def MPI_wall(row):
    material = row['materialwallsext']
    if material in [4, 6, 2, 3, 13]: return 2
    elif material in [1, 5, 8, 9, 11]: return 1
    elif material in [7, 14]: return 0
    else: return None

if 'materialwallsext' in df.columns: df['material_walls'] = df.apply(MPI_wall, axis=1)
if 'materialfloor' in df.columns: df['material_floor'] = df.apply(MPI_floor, axis=1)
if 'materialroof' in df.columns: df['material_roof'] = df.apply(MPI_roof, axis=1)

def average_material_score(row):
    values = [row.get('material_walls'), row.get('material_floor'), row.get('material_roof')]
    valid_values = [v for v in values if v is not None]
    return sum(valid_values) / len(valid_values) if valid_values else None
df['MPI_house'] = df.apply(average_material_score, axis=1)

def update_debtamount(row):
    if row['debt'] == 2.0: return 0
    elif row['debt'] == 1.0: return row['debtamount_1']
    else: return None
if 'debt' in df.columns and 'debtamount_1' in df.columns:
    df['debtamount_1'] = df.apply(update_debtamount, axis=1)

def calculate_school(row, n):
    age_col = f'HHMage_1_{n}'
    school_col = f'HHMschool_{n}'
    schoolcompl_col = f'HHMschoolcompl_{n}'
    if pd.isnull(row.get(age_col)) or row.get(age_col) < 10 or pd.isnull(row.get(school_col)): return None
    elif row.get(school_col) == 2: return 1
    elif pd.isnull(row.get(schoolcompl_col)): return 1
    elif row.get(schoolcompl_col) in [1, -88]: return 1
    else: return 0
for n in range(1, 21):
    if f'HHMage_1_{n}' in df.columns and f'HHMschool_{n}' in df.columns and f'HHMschoolcompl_{n}' in df.columns:
      df[f'MPI_6yearsofschool_perHHM_{n}'] = df.apply(lambda row: calculate_school(row, n), axis=1)

def MPI_6yearsofschool_woman(row):
    if pd.isnull(row.get('school')): return None
    elif row.get('school') == 2: return 1
    elif pd.isnull(row.get('schoolcompleted')): return 1
    elif row.get('schoolcompleted') in [1, -88]: return 1
    else: return 0
if 'school' in df.columns and 'schoolcompleted' in df.columns:
    df['MPI_6yearsofschool_woman'] = df.apply(MPI_6yearsofschool_woman, axis=1)

def MPI_6yearsofschool_allHHM(row):
    education_columns = [col for col in [f'MPI_6yearsofschool_perHHM_{i}' for i in range(1, 21)] + ['MPI_6yearsofschool_woman'] if col in row.index]
    if not education_columns or all(pd.isnull(row[col]) for col in education_columns): return None
    elif any(row[col] == 0 for col in education_columns): return 0
    else: return 1
df['MPI_6yearsofschool_allHHM'] = df.apply(MPI_6yearsofschool_allHHM, axis=1)

if 'occupationmain' in df.columns:
    occupation_scores = {0: 0, 6: 0, 4: 0, 31: 0, 16: 0, 18: 1, 54: 1, 20: 1, 8: 1, 36: 1, 56: 1, 39: 1, 48: 1, 7: 1, 5: 1, 47: 1, 17: 1, 50: 1, 40: 1, 15: 2, 37: 2, 55: 2, 57: 2, 58: 2, 52: 2, 49: 2}
    df['occupation_score'] = df['occupationmain'].map(occupation_scores)

# =============================================================================
# === THIS IS THE CORRECTED FUNCTION ===
# =============================================================================
def map_school_level_to_score(value):
    # Convert value to numeric. If it can't be converted, it becomes NaN.
    value = pd.to_numeric(value, errors='coerce')

    # The rest of the logic now works safely because `value` is either a number or NaN.
    if pd.isnull(value) or value in [-88, 1, 2]:
        return 0  # bad
    elif value in [3, 4]:
        return 1  # medium
    elif value >= 5:
        return 2  # good
    else:
        return 0  # fallback for other numbers or cases

school_columns = [f"HHMschoolcompl_{n}" for n in range(1, 6)] + ["schoolcompleted"]
for col in school_columns:
    if col in df.columns:
        df[f"{col}_score"] = df[col].apply(map_school_level_to_score)

if 'jealousywhat' in df.columns:
    def map_jealousy_score(code):
        if code in [3, 4, 5, 6, 7]: return 0
        elif code in [1, 2, 8]: return 1
        elif code == 0: return 2
        else: return None
    df['jealousywhat_score'] = df['jealousywhat'].apply(map_jealousy_score)

# =============================================================================
# PART 4: MAP REMAINING & BINARY VARIABLES
# =============================================================================
print("\nMapping remaining categorical variables to 0, 1, 2 scores...")

livestock_score_map = {0: 0, '1': 1, '2': 1, '3': 1, '4': 1, '5': 1, '6-10': 2, '11-20': 2, '20+': 2, '21-50': 2, '51+': 2}
livestock_cols = [f"livestocknumbers_{i}" for i in [1, 13, 3, 4, 5, 6, 11, 8, 9, 7, 2, 10]]
for col in livestock_cols:
    if col in df.columns:
        df[col] = df[col].map(livestock_score_map)

if 'debtnote' in df.columns:
    debt_reason_map = {1: 0, 2: 0, 4: 0, 7: 0, 9: 0, 3: 1, 5: 1, 6: 2, 8: 2}
    df['debtnote_score'] = df['debtnote'].map(debt_reason_map)

print("\nMapping binary variables to a 0 (bad) / 1 (good) scale...")
binary_neg = [
    "debt", "foodsecurity1","foodsecurity2", "foodsecurity3", "foodsecurity4", "foodsecurity5",
    "foodsecurity6", "foodsecurity7", "foodsecurity8", "foodsecurity9", "childmortality",
    "jealousy", "assetsmatrix1_4", "assetsmatrix1_5", "assetsmatrix1_22", "assetsmatrix2_15",
    "assetsmatrix2_8", "assetsmatrix3_17", "assetsmatrix2_17",
    "assetsmatrix2_18", "assetsmatrix2_19", "assetsmatrix2_11",
    "assetsmatrix3_15", "assetsmatrix3_23"
]

binary_pos = [
    "HHMschoolnow_1", "HHMschoolnow_2", "HHMschoolnow_3",
    "HHMschoolnow_4", "HHMschoolnow_5",
    "school", "meetings1", "moneywithdraw", "moneyproblems"
]

neg_map = {1.0: 0.0, 2.0: 1.0}
for col in binary_neg:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df[col].map(neg_map)
        print(f"Mapped '{col}' (negative binary) to 0/1.")

pos_map = {1.0: 1.0, 2.0: 0.0}
for col in binary_pos:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df[col].map(pos_map)
        print(f"Mapped '{col}' (positive binary) to 0/1.")


# =============================================================================
# PART 5: SAVE TO EXCEL
# =============================================================================
output_filename = 'processed_data_with_scores.xlsx'
print(f"\nSaving the processed DataFrame to '{output_filename}'...")

try:
    with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='Processed_Data', index=False)
    print("File saved successfully.")
except ImportError:
    print("Warning: 'xlsxwriter' is not installed. Trying with default engine.")
    try:
        df.to_excel(output_filename, index=False)
        print("File saved successfully with default engine.")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")
except Exception as e:
    print(f"An error occurred while saving the file: {e}")

/tmp/ipython-input-1-1136453054.py:16: DtypeWarning: Columns (12,29,36,39,41,48,57,83,84,85,86,87,88,89,90,91,92,435,438,457,466,472,474,479,480,483,487,488,493,496,497,499,500,541,561,563,586,587,591,592,593,597,599,601,602,621,622,624,635,641,679) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/central-tableau-export-2.0.csv')
/tmp/ipython-input-1-1136453054.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Round'] = df['Round'].astype(int)
/tmp/ipython-input-1-1136453054.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Initial 'Round' values found: ['0' '100' '102' '2' '1' '3']

Mapping remaining categorical variables to 0, 1, 2 scores...

Mapping binary variables to a 0 (bad) / 1 (good) scale...
Mapped 'debt' (negative binary) to 0/1.
Mapped 'foodsecurity1' (negative binary) to 0/1.
Mapped 'foodsecurity2' (negative binary) to 0/1.
Mapped 'foodsecurity3' (negative binary) to 0/1.
Mapped 'foodsecurity4' (negative binary) to 0/1.
Mapped 'foodsecurity5' (negative binary) to 0/1.
Mapped 'foodsecurity6' (negative binary) to 0/1.
Mapped 'foodsecurity7' (negative binary) to 0/1.
Mapped 'foodsecurity8' (negative binary) to 0/1.
Mapped 'foodsecurity9' (negative binary) to 0/1.
Mapped 'childmortality' (negative binary) to 0/1.
Mapped 'jealousy' (negative binary) to 0/1.
Mapped 'assetsmatrix1_4' (negative binary) to 0/1.
Mapped 'assetsmatrix1_5' (negative binary) to 0/1.
Mapped 'assetsmatrix1_22' (negative binary) to 0/1.
Mapped 'assetsmatrix2_15' (negative binary) to 0/1.
Mapped 'assetsmatrix2_8' (negative bin

In [9]:
# Load the data
df = pd.read_excel('processed_data_with_scores.xlsx')

# --- Variable Definitions ---
numerical = [
    "savinghowmuch_1", "savinghowmuch_2", "savinghowmuch_3",
    "savingstotal_1", "debtamount_1", "timewatersource_1"
]
ordered_categorical = [
    *[f"foodsecurity{i}freq" for i in range(1, 10)],
    *[f"anxiety{i}" for i in range(1, 8)],
    "psychwellbeing_1", "psychwellbeing_3", "psychwellbeing_5", "psychwellbeing2_5"
]
categorical = [
    "fuelcooking", "sourcelighting", "watersource", "Toiletfacility",
    "materialroof", "materialfloor", "materialwallsext",
    *[f"HHMschoolcompl_{n}" for n in range(1, 6)],
    "schoolcompleted", "livestocknumbers_1",
    *[f"livestocknumbers_{i}" for i in [1, 13, 3, 4, 5, 6, 11, 8, 9, 7, 2, 10]],
    "occupationmain"
]
binary = [
    "childmortality",
    *[f"foodsecurity{i}" for i in range(1, 10)],
    *[f"HHMschool_{n}" for n in range(1, 6)],
    *[f"HHMschoolnow_{n}" for n in range(1, 6)],
    "school", "debt", "jealousy",
    "assetsmatrix1_4", "assetsmatrix1_5", "assetsmatrix1_22",
    "assetsmatrix2_15", "assetsmatrix2_8", "assetsmatrix3_17",
    "assetsmatrix2_17", "assetsmatrix2_18", "assetsmatrix2_19",
    "assetsmatrix2_11", "assetsmatrix3_15", "assetsmatrix3_23",
    "meetings1", "moneywithdraw", "moneyproblems"
]
multiple_choice = ["debtnote", "jealousywhat"]
information = ["Country", "Groupnr", "Round"]


# --- Benchmark Calculation Function (As Provided) ---
def calculate_benchmarks(df: pd.DataFrame) -> pd.DataFrame:
    """Calculates benchmark statistics (mean/proportion) for all variables per round."""
    round_to_benchmark_map = {0: "benchmark_baseline", 1: "benchmark_phone_survey_1", 2: "benchmark_phone_survey_2",
                              3: "benchmark_phone_survey_3", 100: "benchmark_endline",
                              102: "benchmark_post-program_survey_2"}
    binary_neg = ["debt", *[f"foodsecurity{i}" for i in range(1, 10)], "childmortality", "jealousy", "assetsmatrix1_4",
                  "assetsmatrix1_5", "assetsmatrix1_22", "assetsmatrix2_15", "assetsmatrix2_8", "assetsmatrix3_17",
                  "assetsmatrix2_17", "assetsmatrix2_18", "assetsmatrix2_19", "assetsmatrix2_11", "assetsmatrix3_15",
                  "assetsmatrix3_23"]
    binary_pos = [*[f"HHMschoolnow_{n}" for n in range(1, 6)], "school", "meetings1", "moneywithdraw", "moneyproblems"]

    df_work = df.copy()
    df_work['Round'] = pd.to_numeric(df_work['Round'], errors='coerce')
    df_work.dropna(subset=['Round'], inplace=True)
    df_work['Round'] = df_work['Round'].astype(int)
    df_work = df_work[df_work['Round'].isin(round_to_benchmark_map.keys())]

    all_benchmark_cols = list(set(numerical + ordered_categorical + binary))
    for col in all_benchmark_cols:
        if col in df_work.columns: df_work[col] = pd.to_numeric(df_work[col], errors='coerce')

    agg_dict = {}
    for col in list(set(numerical + ordered_categorical)):
        if col in df_work.columns: agg_dict[col] = 'mean'
    for col in binary_pos:
        if col in df_work.columns: agg_dict[col] = lambda s: (s == 1).sum() / s.isin(
            [1, 2]).sum() if s.isin([1, 2]).sum() > 0 else np.nan
    for col in binary_neg:
        if col in df_work.columns: agg_dict[col] = lambda s: (s == 2).sum() / s.isin(
            [1, 2]).sum() if s.isin([1, 2]).sum() > 0 else np.nan

    benchmarks_df = df_work.groupby('Round').agg(agg_dict)
    benchmarks_df = benchmarks_df.rename(index=round_to_benchmark_map)

    ordered_cols = [c for c in numerical + ordered_categorical + binary_pos + binary_neg if c in benchmarks_df.columns]
    return benchmarks_df[ordered_cols]


# --- Plotting & Data Prep Functions (Unchanged) ---
def plot_groups_vs_overall(df, groups, variables, dpi=120):
    round_order = [0, 1, 2, 3, 100, 102]
    labels = {0: 'Baseline', 1: 'Phone survey 1', 2: 'Phone survey 2', 3: 'Phone survey 3', 100: 'Endline',
              102: 'Post-program survey'}
    df = df.copy();
    df['Round'] = df['Round'].astype(int)
    plt.style.use('ggplot');
    cmap = plt.colormaps['tab10']
    buffers = {}
    for var in variables:
        if var not in df.columns or df[var].dropna().empty: continue
        fig, ax = plt.subplots(figsize=(10, 6), dpi=dpi)
        overall_stats = df.groupby('Round')[var].agg(mean='mean', q1=lambda x: x.quantile(0.25),
                                                     q3=lambda x: x.quantile(0.75)).reindex(round_order).reset_index()
        overall_stats['lbl'] = overall_stats['Round'].map(labels)
        overall_stats['iqr'] = overall_stats['q3'] - overall_stats['q1']
        overall_stats['lower_bound'] = overall_stats['q1'] - 1.5 * overall_stats['iqr']
        overall_stats['upper_bound'] = overall_stats['q3'] + 1.5 * overall_stats['iqr']
        x_positions = np.arange(len(round_order))
        ax.fill_between(x_positions, overall_stats['lower_bound'], overall_stats['upper_bound'], color='lightgray',
                        alpha=0.6, label='Overall IQR ±1.5')
        ax.plot(x_positions, overall_stats['lower_bound'], 'k-', lw=0.8);
        ax.plot(x_positions, overall_stats['upper_bound'], 'k-', lw=0.8)
        ax.plot(x_positions, overall_stats['mean'], linestyle='--', marker='o', color='tab:red', linewidth=2,
                markersize=8, label='Overall average')
        for i, grp in enumerate(groups):
            sub = df[df['Groupnr'] == grp][['Round', var]].dropna()
            if sub.empty: continue
            pts = sorted(zip(sub['Round'], sub[var]), key=lambda x: round_order.index(x[0]))
            rounds, vals = zip(*pts)
            x_group_positions = [round_order.index(r) for r in rounds]
            for j in range(1, len(rounds)):
                idx0, idx1 = round_order.index(rounds[j - 1]), round_order.index(rounds[j])
                style = '-' if idx1 - idx0 == 1 else 'dotted'
                ax.plot(x_positions[[idx0, idx1]], [vals[j - 1], vals[j]], linestyle=style, color=cmap(i),
                        linewidth=1.5)
            ax.scatter(x_group_positions, vals, marker='s', s=50, color=cmap(i), label=f'Group {grp}')
        ax.set_title(f'{var} trend for groups vs overall (with IQR band)', fontsize=16, pad=12)
        ax.set_xlabel('Survey Round', fontsize=14, labelpad=8);
        ax.set_ylabel(var, fontsize=14, labelpad=8)
        ax.set_xticks(x_positions);
        ax.set_xticklabels([labels[r] for r in round_order], rotation=45, ha='right')
        ax.grid(alpha=0.3);
        ax.spines['top'].set_visible(False);
        ax.spines['right'].set_visible(False)
        ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1));
        plt.tight_layout()
        buf = io.BytesIO();
        fig.savefig(buf, format='png', bbox_inches='tight');
        plt.close(fig);
        buf.seek(0)
        buffers[var] = buf
    return buffers


def growth_plot(df: pd.DataFrame, groups: list[str], var: str, window: int = 3) -> io.BytesIO:
    """
    Generates a plot showing growth outliers for specified groups against the overall trend.
    This version ensures that lines connecting a group's data points are always solid.
    """
    round_order = [0, 1, 2, 3, 100, 102]
    labels = {0: 'Baseline', 1: 'Phone survey 1', 2: 'Phone survey 2', 3: 'Phone survey 3', 100: 'Endline',
              102: 'Post-program survey'}
    x = np.arange(len(round_order))
    xlabs = [labels[r] for r in round_order]

    ewma_col = f"{var}_EWMA_{window}_growth"
    if ewma_col not in df.columns:
        return None

    stats = df.groupby('Round')[ewma_col].agg(
        Q1=lambda x: x.quantile(0.25),
        Q3=lambda x: x.quantile(0.75)
    ).reindex(round_order)

    iqr = stats['Q3'] - stats['Q1']
    lower = stats['Q1'] - 1.5 * iqr
    upper = stats['Q3'] + 1.5 * iqr
    raw_gc = f"{var}_growth"
    overall = df.groupby('Round')[raw_gc].mean().reindex(round_order)

    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize=(10, 5), dpi=120)

    # Plot Overall Stats (IQR Band and Average Growth)
    ax.fill_between(x, lower, upper, color='lightgray', alpha=0.6, label='EWMA IQR ±1.5')
    ax.plot(x, lower, 'k-', lw=1)
    ax.plot(x, upper, 'k-', lw=1)
    ax.plot(x, overall.values, 'o--', color='red', lw=2, ms=6, label='Avg growth')

    # Setup legend for groups
    cmap = plt.get_cmap('tab10')
    for i, gid in enumerate(groups):
        ax.plot([], [], color=cmap(i % 10), marker='s', linestyle='-', linewidth=1.5,
                label=f"Group {gid}")

    # --- CORRECTED PLOTTING LOGIC FOR GROUPS ---
    def plot_group(df_group, color):
        """Plots a single group's data with solid lines."""
        # Get the y-values, reindexing to ensure alignment with all possible rounds
        y = (df_group.set_index('Round')[raw_gc].reindex(round_order).values)

        # Filter out rounds where this group has no data
        valid_indices = ~np.isnan(y)
        x_valid, y_valid = x[valid_indices], y[valid_indices]

        # Plot the valid points with a solid connecting line
        ax.plot(x_valid, y_valid, color=color, lw=1.5, linestyle='-')
        ax.scatter(x_valid, y_valid, color=color, s=60, marker='s')

    # Iterate through specified groups and plot their data
    for i, gid in enumerate(groups):
        df_group = df[df['Groupnr'] == gid].copy()
        if not df_group.empty and raw_gc in df_group.columns:
            plot_group(df_group, cmap(i % 10))

    # Final plot styling
    ax.set_xticks(x)
    ax.set_xticklabels(xlabs, rotation=45, ha='right')
    ax.set_ylabel(f"{var}_growth (Δ)")
    ax.set_title(f"Growth outliers: {var}")
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1))
    plt.tight_layout()

    # Save plot to buffer
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    return buf


def add_ewma_columns(df: pd.DataFrame, variables: list[str] | None = None, span: int = 3, adjust: bool = True,
                     min_periods: int = 1) -> pd.DataFrame:
    df_ewma = df.copy()
    if variables is None: variables = [col for col in df_ewma.select_dtypes(include='number').columns.tolist() if
                                       col != 'Round']
    for var in variables: df_ewma[f"{var}_EWMA_{span}"] = df_ewma.groupby('Groupnr')[var].transform(
        lambda s: s.ewm(span=span, adjust=adjust, min_periods=min_periods).mean())
    return df_ewma


def add_growth_to_smoothed_with_interpolation(df: pd.DataFrame, col: str, group_key: str = 'Groupnr') -> pd.Series:
    interp = df.groupby(group_key)[col].transform(lambda s: s.interpolate(method='linear', limit_area=None, limit=2))
    return interp.groupby(df[group_key]).diff()


def df_prep(df: pd.DataFrame, variables: list[str]) -> pd.DataFrame:
    required_cols = ['Groupnr', 'Country', 'Round'] + variables
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        variables = [col for col in variables if col not in missing_cols]
        required_cols = ['Groupnr', 'Country', 'Round'] + variables
    df2 = df[required_cols].copy()
    df2 = df2[~df2['Round'].isin(['Onboarding', '6', '6.0']) & df2['Round'].notna()]
    df2['Round'] = pd.to_numeric(df2['Round'], errors='coerce')
    df2 = df2[df2['Round'] % 1 == 0];
    df2['Round'] = df2['Round'].astype(int)
    df2 = df2[df2['Round'].isin([0, 1, 2, 3, 100, 102])].sort_values('Round').reset_index(drop=True)
    numeric_like_vars = list(
        dict.fromkeys([v for v in variables if v not in ['Groupnr', 'Country', 'Round', 'livestocknumbers_1']]))
    for col in numeric_like_vars:
        if col in df2.columns: df2[col] = pd.to_numeric(df2[col], errors='coerce')
    df_tr = df2.groupby(['Groupnr', 'Round'])[numeric_like_vars].mean().reset_index().sort_values(
        ['Groupnr', 'Round']).reset_index(drop=True).copy()
    df_tr = add_ewma_columns(df_tr, variables=numeric_like_vars, span=3, min_periods=1)
    new_cols_data = {}
    for var in numeric_like_vars:
        col = f'{var}_EWMA_3'
        if col in df_tr.columns: new_cols_data[f'{col}_growth'] = add_growth_to_smoothed_with_interpolation(df_tr, col)
    for var in numeric_like_vars:
        new_cols_data[f'{var}_growth'] = df_tr.groupby('Groupnr')[var].diff()
        if var in df_tr.columns: new_cols_data[f'{var}_pct_growth'] = df_tr.groupby('Groupnr')[var].pct_change(
            fill_method=None) * 100
    if new_cols_data: df_tr = pd.concat([df_tr, pd.DataFrame(new_cols_data, index=df_tr.index)], axis=1)
    df_tr = df_tr.sort_values(['Groupnr', 'Round']).reset_index(drop=True)
    return df_tr


def final_prep(raw_df, variables, groups, country, window=3):
    df_all = df_prep(raw_df, variables)
    df_all = df_all.drop_duplicates(subset=['Groupnr', 'Round'])
    if country: df_all = df_all[df_all['Groupnr'].str.startswith(country)]
    iqr_bands, avg_growth = {}, {}
    for var in variables:
        raw_gc, ewma_gc = f"{var}_growth", f"{var}_EWMA_{window}_growth"
        if ewma_gc not in df_all.columns or raw_gc not in df_all.columns: continue
        if df_all[ewma_gc].dropna().empty:
            lower, upper = pd.Series(np.nan), pd.Series(np.nan)
        else:
            q1_ewma, q3_ewma = df_all.groupby('Round')[ewma_gc].quantile(0.25), df_all.groupby('Round')[
                ewma_gc].quantile(0.75)
            iqr_ewma = q3_ewma - q1_ewma
            lower, upper = q1_ewma - 1.5 * iqr_ewma, q3_ewma + 1.5 * iqr_ewma
        iqr_bands[var] = pd.DataFrame({'lower': lower, 'upper': upper})
        if df_all[raw_gc].dropna().empty:
            avg_growth[var] = pd.Series(np.nan)
        else:
            avg_growth[var] = df_all.groupby('Round')[raw_gc].mean()
    df_filt = df_all[df_all['Groupnr'].isin(groups)]
    flags = {var: {} for var in variables}
    for var in variables:
        if var not in iqr_bands or var not in avg_growth: continue
        band = iqr_bands[var].reset_index().rename(columns={'index': 'Round'})
        raw_gc = f"{var}_growth"
        if raw_gc not in df_filt.columns: continue
        tmp = df_filt[['Groupnr', 'Round', raw_gc]].merge(band, on='Round', how='left')
        for _, r in tmp.iterrows():
            v = r[raw_gc]
            if pd.isna(v) or pd.isna(r['lower']) or pd.isna(r['upper']): continue
            if v < r['lower']:
                flags[var][(r['Groupnr'], r['Round'])] = 'NEG'
            elif v > r['upper']:
                flags[var][(r['Groupnr'], r['Round'])] = 'POS'
    rows = [{'Groupnr': grp, 'Round': rnd, 'variable': var, 'flag': tag} for var, vf in flags.items() for
            (grp, rnd), tag in vf.items()]
    df_flags = pd.DataFrame(rows)
    if not df_flags.empty:
        df_out = df_flags.pivot(index=['Groupnr', 'Round'], columns='variable', values='flag').reset_index()
    else:
        df_out = pd.DataFrame(columns=['Groupnr', 'Round'] + variables)
    return df_out, flags, df_all


# --- NEW: Function to calculate outliers based on static values within a round ---
def calculate_round_outliers(df: pd.DataFrame, num_bin_vars: list[str], cat_vars: list[str]) -> pd.DataFrame:
    """
    Calculates outliers based on static values within each round using the IQR method.
    - For numerical/binary variables, it uses the group's mean.
    - For categorical variables, it uses the percentage for each category.
    """
    all_outliers = []
    df_work = df.copy()
    df_work['Round'] = pd.to_numeric(df_work['Round'], errors='coerce').astype('Int64')

    # Ensure relevant columns are numeric
    for col in num_bin_vars:
        if col in df_work.columns:
            df_work[col] = pd.to_numeric(df_work[col], errors='coerce')

    rounds = sorted(df_work['Round'].dropna().unique())

    # --- Part 1: Numerical and Binary Variables ---
    for var in num_bin_vars:
        if var not in df_work.columns: continue

        # Calculate group means for the variable across all rounds
        group_means = df_work.groupby(['Round', 'Groupnr'])[var].mean().reset_index()

        for r in rounds:
            round_means = group_means[group_means['Round'] == r][var]
            if len(round_means) < 4: continue  # Not enough data for IQR

            Q1 = round_means.quantile(0.25)
            Q3 = round_means.quantile(0.75)
            IQR = Q3 - Q1
            if IQR == 0: continue

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers = group_means[
                (group_means['Round'] == r) &
                ((group_means[var] < lower_bound) | (group_means[var] > upper_bound))
                ]

            for _, row in outliers.iterrows():
                flag = "HIGH" if row[var] > Q3 else "LOW"
                description = f"{var} ({flag} mean: {row[var]:.2f})"
                all_outliers.append({
                    'Groupnr': row['Groupnr'], 'Round': r, 'flag_description': description
                })

    # --- Part 2: Categorical Variables ---
    for var in cat_vars:
        if var not in df_work.columns or df_work[var].nunique() < 2: continue

        for r in rounds:
            df_round = df_work[df_work['Round'] == r].dropna(subset=[var])
            if df_round.empty: continue

            # Calculate percentage of each category per group
            cross_tab = pd.crosstab(index=df_round['Groupnr'], columns=df_round[var], normalize='index') * 100
            if cross_tab.shape[0] < 4: continue

            # Check each category for outliers
            for category in cross_tab.columns:
                cat_percentages = cross_tab[category]

                Q1 = cat_percentages.quantile(0.25)
                Q3 = cat_percentages.quantile(0.75)
                IQR = Q3 - Q1
                if IQR == 0: continue

                # We are primarily interested in unusually high percentages
                upper_bound = Q3 + 1.5 * IQR

                outliers = cat_percentages[cat_percentages > upper_bound]

                for group, pct_val in outliers.items():
                    description = f"{var}: {category} (HIGH %: {pct_val:.0f}%)"
                    all_outliers.append({
                        'Groupnr': group, 'Round': r, 'flag_description': description
                    })

    if not all_outliers:
        return pd.DataFrame()
    return pd.DataFrame(all_outliers)


# --- FINALIZED: xlsx file generator with clustering instructions ---
def xlsx_generator(raw_df, variables, country, groups, window=3):
    print("Step 1: Preparing data and identifying trend outliers...")

    all_vars = variables

    vars_to_exclude = (
    [f"foodsecurity{i}freq" for i in range(1, 10)] +
    [f"foodsecurity{i}" for i in range(1, 10)] +
    [f"livestocknumbers_{i}" for i in [1, 13, 3, 4, 5, 6, 11, 8, 9, 7, 2, 10]] +
    [f"HHMschool_{n}" for n in range(1, 6)] +
    [f"HHMschoolnow_{n}" for n in range(1, 6)] +
    [f"HHMschoolcompl_{n}" for n in range(1, 6)] +
    ["assetsmatrix1_4", "assetsmatrix1_5", "assetsmatrix1_22",
    "assetsmatrix2_15", "assetsmatrix2_8", "assetsmatrix3_17",
    "assetsmatrix2_17", "assetsmatrix2_18", "assetsmatrix2_19",
    "assetsmatrix2_11", "assetsmatrix3_15", "assetsmatrix3_23"],
    ['meetings1']
    )


    trend_vars = [var for var in all_variables if var not in vars_to_exclude]

    _, flags, df_all = final_prep(raw_df, trend_vars, groups, country, window)

    print("Step 2: Calculating benchmark statistics...")
    country_df = raw_df[raw_df['Country'] == country].copy()
    df_benchmarks = calculate_benchmarks(country_df)

    print("Step 2b: Calculating round-based outliers...")
    num_bin_vars_for_calc = list(set(numerical + binary))
    num_bin_vars_for_calc = [v for v in num_bin_vars_for_calc if v in country_df.columns]
    cat_vars_for_calc = [v for v in categorical if v in country_df.columns]
    df_round_outliers = calculate_round_outliers(country_df, num_bin_vars_for_calc, cat_vars_for_calc)

    # --- Setup Excel Writer ---
    filename = f"Analysis by Group for {country}.xlsx"
    writer = pd.ExcelWriter(filename, engine="xlsxwriter")
    workbook = writer.book
    bold = workbook.add_format({"bold": True})
    plot_title_format = workbook.add_format({'bold': True, 'font_size': 11})
    wrap_format = workbook.add_format({'text_wrap': True, 'valign': 'top'})
    url_format = workbook.add_format({'font_color': 'blue', 'underline': 1})

    # --- Writing Summary Sheets ---
    print("Step 3: Writing summary sheets to Excel...")

    # --- REVISED: Add the Clustering sheet with the hyperlink and instructions ---
    clustering_ws = workbook.add_worksheet("clustering")

    # Define your instructional text here. You can paste your final text inside the triple quotes.
    instructional_text = """Instructions for the Clustering Application :

Introduction
This application was developed for 100WEEKS to compare the performance of participating groups
of women during the program. With the application, vulnerable groups can be identified and
additional support can be provided.

Access to the Application
https://100weeks-clustering.streamlit.app/
The application is developed to be user-friendly and it does not require any technical background to
operate. After opening the application, a specific country can be selected via the drop-down menu.
Based on this selection, visualizations are generated for the different survey rounds within that
country. Plots are visualized for the baseline, round 2 (after 50 weeks), and round 100 (after 100
weeks).

Using the Application
Every plot shows the results of a specific survey round. Each data point in the plot represents a
participating group of women in the 100WEEKS program. By moving the computer mouse over a
data point, the corresponding group number is displayed. In addition, a search bar is included for each
plot where you can manually search for a specific group (e.g. GHA001). The group will appear in the
plot, marked with a cross. The bottom left corner of the plots usually contains the most vulnerable
groups, which score relatively low on poverty-related indicators. The top right corner shows groups
that do relatively well on these indicators. It is important to mention that the plots are based on a
simplified representation of the data. The dataset with multiple dimensions is reduced to two principal
dimensions using Principal Component Analysis (PCA). This ensures that the results can be
visualized in two-dimensional plots, but the plots only show an approximation of reality.

Technical Setup
The complete code and dataset are stored in the following GitHub repository:
https://github.com/sam-jager/100weeks-clustering. The clustering.py file contains all of the python
code. The file requirements.txt contains the Python libraries that are needed for the application to run.

Adding a New Dataset
When a new dataset is available for analysis, this can be uploaded with these steps. First, click the link
to open the 100weeks-clustering GitHub repository. Click on the button ‘Add file’ and upload the new
dataset. After this, open the clustering.py file and search for the following line of code at the
beginning of the file:
url = "https://raw.githubusercontent.com/sam-jager/100weeks-clustering/main/central-tableau-export-2.0.csv".
Replace the filename (central-tableau-export-2.0.csv) in this URL with the new dataset’s filename.
Save the changes with the ‘Commit changes’ button. The application automatically uses the new
dataset.
"""

    # Set column width to be wide enough for instructions
    clustering_ws.set_column('A:A', 80)

    # Write the hyperlink to cell A1
    clustering_ws.write_url(
        'A1',
        'https://100weeks-clustering.streamlit.app/',
        url_format,
        'Link to the Clustering Application'
    )

    # Write the instructional text to cell A3, applying the wrap format
    clustering_ws.write('A3', instructional_text, wrap_format)

    # Sheet: Trend-based Outlier Summary
    summary_ws = workbook.add_worksheet("growth_outlier_summary")
    all_flags_rows = [
        {'Groupnr': g, 'variable': var, 'Round': rnd}
        for var, vf in flags.items()
        for (g, rnd), tag in vf.items()
    ]
    if not all_flags_rows:
        summary_ws.write('A1', 'No trend-based outliers found to summarize.')
    else:
        df_flags_long = pd.DataFrame(all_flags_rows)
        df_grouped = (
            df_flags_long.groupby(['Groupnr', 'Round'])['variable']
            .apply(lambda x: '\n'.join(sorted(x)))
            .reset_index()
        )
        df_outlier_summary = df_grouped.pivot(
            index='Groupnr', columns='Round', values='variable'
        )
        desired_round_order = [0, 1, 2, 3, 100, 102]
        df_outlier_summary = df_outlier_summary.reindex(columns=desired_round_order).fillna('').reset_index()
        summary_ws.set_column('A:A', 15);
        summary_ws.set_column('B:G', 30)
        for col_num, value in enumerate(df_outlier_summary.columns.values):
            summary_ws.write(0, col_num, value, bold)
        for row_num, row_data in enumerate(df_outlier_summary.itertuples(index=False), start=1):
            summary_ws.write(row_num, 0, row_data[0])
            for col_num, cell_data in enumerate(row_data[1:], start=1):
                summary_ws.write(row_num, col_num, cell_data, wrap_format)

    # Sheet: Round-based Outlier Summary
    round_summary_ws = workbook.add_worksheet("round_outlier_summary")
    if df_round_outliers.empty:
        round_summary_ws.write('A1', 'No round-based outliers found to summarize.')
        print("No round-based outliers found.")
    else:
        df_round_grouped = (
            df_round_outliers.groupby(['Groupnr', 'Round'])['flag_description']
            .apply(lambda x: '\n'.join(sorted(x)))
            .reset_index()
        )
        df_round_pivot = df_round_grouped.pivot(
            index='Groupnr', columns='Round', values='flag_description'
        )
        desired_round_order = [0, 1, 2, 3, 100, 102]
        df_round_pivot = df_round_pivot.reindex(columns=desired_round_order).fillna('').reset_index()
        round_summary_ws.set_column('A:A', 15);
        round_summary_ws.set_column('B:G', 40)
        for col_num, value in enumerate(df_round_pivot.columns.values):
            round_summary_ws.write(0, col_num, value, bold)
        for row_num, row_data in enumerate(df_round_pivot.itertuples(index=False), start=1):
            round_summary_ws.write(row_num, 0, row_data[0])
            for col_num, cell_data in enumerate(row_data[1:], start=1):
                round_summary_ws.write(row_num, col_num, cell_data, wrap_format)

    # Sheet: Benchmark Summary
    df_benchmarks.to_excel(writer, sheet_name="benchmark_summary", index=True)

    # --- EXISTING: Write Per-Group Sheets (Based on trend outliers) ---
    print(f"Step 4: Generating individual sheets for flagged groups...")
    for grp in groups:
        rows = [{"Round": rnd, "variable": var, "flag": tag} for var, vf in flags.items() for (g, rnd), tag in
                vf.items() if g == grp]
        if not rows: continue
        df_grp = pd.DataFrame(rows)
        df_wide = df_grp.pivot(index="Round", columns="variable", values="flag").reset_index().replace(
            [np.inf, -np.inf, np.nan], None)
        ws = workbook.add_worksheet(grp)
        for c, col in enumerate(df_wide.columns): ws.write(0, c, col, bold)
        for r, rec in enumerate(df_wide.itertuples(index=False), start=1):
            for c, val in enumerate(rec): ws.write(r, c, val)
        rowpos = len(df_wide) + 2
        vars_flagged = sorted(df_grp["variable"].unique())
        for var in vars_flagged:
            ws.write(rowpos, 0, f"Analysis for variable: {var}", plot_title_format);
            rowpos += 2
            buf1 = growth_plot(df_all, [grp], var, window=window)
            if buf1: ws.insert_image(rowpos, 0, "", {"image_data": buf1, "x_scale": 0.75, "y_scale": 0.75})
            buf2 = plot_groups_vs_overall(df_all, [grp], [var]).get(var)
            if buf2: ws.insert_image(rowpos, 12, "", {"image_data": buf2, "x_scale": 0.75, "y_scale": 0.75})
            rowpos += 22

    writer.close()
    print(f"✅ Success! Workbook saved to {filename}")


# --- Main Execution ---
all_variables = numerical + ordered_categorical + binary + categorical
raw_df = df.copy()
variables = all_variables
# country = 'GHA'
# groups = [f'GHA{i:03d}' for i in range(1, 11)]
groups = raw_df[raw_df['Country'] == country]['Groupnr'].unique()
xlsx_generator(raw_df, variables, country, groups, 3)

KeyboardInterrupt: 